In [3]:
import os
import sys
#import pandas as pd
import numpy as np
#from preprocess import clean_abstract, preprocess_record
from encoder import embed_texts

c:\Users\Alonso\.conda\envs\specter-gpu\lib\site-packages\torch\_subclasses\functional_tensor.py:275: UserWarning: Failed to initialize NumPy: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado. (Triggered internally at C:\b\abs_34s6o8i12c\croot\libtorch_1751464457133\work\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))
c:\Users\Alonso\.conda\envs\specter-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy._core.multiarray failed to import

In [2]:
# 1) Cargar datos
df = pd.read_excel("data_concatenada.xlsx",
                   usecols=["Código VRID", "Título", "Resumen", "Keywords", "Interdisciplinario", "Transdisciplinario"]) \
       .fillna("")
cols = ["Título", "Resumen", "Keywords"]
df[cols] = df[cols].applymap(lambda x: "" if str(x).strip().upper() == "DESCONOCIDO" else str(x).strip())
df["text_for_embedding"] = df.apply(
    lambda r: preprocess_record(r["Título"], r["Resumen"], r["Keywords"]),
    axis=1
)
#df.to_excel("vis_exequiel.xlsx", index=False)

C:\Users\Alonso\AppData\Local\Temp\ipykernel_10244\399713410.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols] = df[cols].applymap(lambda x: "" if str(x).strip().upper() == "DESCONOCIDO" else str(x).strip())


In [ ]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel
import torch

# Parámetros generales
BASE_MODEL = "allenai/specter2_base"
ADAPTER_NAME = "allenai/specter2"
BATCH_SIZE = 16
MIN_SIMILARITY_PRIMARY = 0.80
RELATIVE_THRESHOLD = 0.90
MIN_SIMILARITY_SECOND = 0.85
DISCIPLINE_ALIGNMENT_THRESHOLD = 0.1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3) Cargar modelo SPECTER2
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
model = AutoAdapterModel.from_pretrained(BASE_MODEL, trust_remote_code=True)
model.load_adapter(ADAPTER_NAME, source="hf", set_active=True)
model.to(device)
model.eval()
print("Modelo SPECTER2 cargado correctamente.")

text_embeddings = embed_texts(
    df["text_for_embedding"].tolist(),
    model=None,  # Replace with your BERT model